In [20]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from pprint import pprint

# sen_df: pd.DataFrame = pd.read_csv('/media/ponbac/BigHDD/ethereum/blacklist/seniority/seniority-tornado-result.csv', index_col=0)
# poison_df: pd.DataFrame = pd.read_csv('/media/ponbac/BigHDD/ethereum/blacklist/poison/poison-tornado-result.csv', index_col=0)
# haircut_df: pd.DataFrame = pd.read_csv('/media/ponbac/BigHDD/ethereum/blacklist/haircut/haircut-tornado-result.csv', index_col=0)
# fifo_df: pd.DataFrame = pd.read_csv('/media/ponbac/BigHDD/ethereum/blacklist/fifo/fifo-tornado-result.csv', index_col=0)

## SQL Setup

In [21]:
from sqlalchemy.orm import Session
from db.sql import crud, schemas
from db.sql.database import SessionLocal

def get_db():
    db = SessionLocal()
    return db

## Mongo Setup

In [22]:
from db.mongo.database import IndagoSession
from db.mongo.schemas import DARAddressMapping, DARGraph

# MONGO
db = IndagoSession
dar_collection = db['dar']
dar_map_collection = db['dar_map']

def is_flagged(address: str) -> bool:
    """
    Check if an address is flagged in the blacklist.

    TODO: Should not only check seniority, but some kind of combined blacklisting mechanism.
    """
    db = SessionLocal()
    tainted = crud.get_seniority(db, address=address)
    db.close()

    return True if tainted is not None else False

async def fetch_cluster(address: str) -> DARGraph:
    address = address.lower()
    cluster_id: int = None
    if (id := await dar_map_collection.find_one({"_id": address})) is not None:
        cluster_id = id["cluster_id"]
    else:
        print(f"Graph for {address} not found")
        return None
    if (graph := await dar_collection.find_one({"_id": cluster_id})) is not None:
        # TODO: Should be done somewhere else, this is not a good place for it

        # Flags the non-exchange addresses if they are in the blacklist
        non_exchange_nodes = []
        exchange_nodes = []
        for edge in graph['edges']:
            non_exchange_nodes.append(graph['nodes'][edge[0]])
            exchange_nodes.append(graph['nodes'][edge[1]])
        non_exchange_nodes = set(non_exchange_nodes)
        exchange_nodes = set(exchange_nodes)
        
        checked_nodes = []
        for node in non_exchange_nodes:
            if is_flagged(node):
                checked_nodes.append({"address": node, "flagged": True})
            else:
                checked_nodes.append({"address": node, "flagged": False})
        for node in exchange_nodes:
            checked_nodes.append({"address": node, "flagged": False})

        graph['nodes'] = checked_nodes
        # pprint(graph['nodes'])

        # Converts the edge-objects to a dict instead of a simple array
        parsed_edges = []
        for edge in graph["edges"]:
            parsed_edges.append({"from": edge[0], "to": edge[1]})
        graph["edges"] = parsed_edges

        return graph
    print(f"Graph {id} not found")
    return None

# Fetch results

In [23]:
def wei_to_eth(wei: int) -> float:
    return wei / 10**18

def get_combined_blacklisting_data(address: str):
    db = SessionLocal()
    sen = crud.get_seniority(db, address=address)
    poison = crud.get_poison(db, address=address)
    haircut = crud.get_haircut(db, address=address)
    fifo = crud.get_fifo(db, address=address)
    db.close()

    return {
        "address": address,
        "seniority": wei_to_eth(sen.tainted_balance) if sen is not None else 0,
        "poison": True if poison is not None else False,
        "haircut": wei_to_eth(haircut.tainted_balance) if haircut is not None else 0,
        "fifo": wei_to_eth(fifo.tainted_balance) if fifo is not None else 0,
    }

async def get_cluster_with_blacklisting_data(address: str):
    cluster = await fetch_cluster(address)
    if cluster is None:
        return None
    cluster['blacklisting'] = [get_combined_blacklisting_data(node['address']) for node in cluster['nodes']]
    return cluster

In [25]:
data = await get_cluster_with_blacklisting_data('0x2f86e77d4e577d32f5d1d51289f80cfc95b4f460')
pprint(data)

{'_id': 34,
 'blacklisting': [{'address': '0x2f86e77d4e577d32f5d1d51289f80cfc95b4f460',
                   'fifo': 0,
                   'haircut': 0,
                   'poison': False,
                   'seniority': 0},
                  {'address': '0x5a6e548d1b18f3826dc154df8ff87be0f49ba1f9',
                   'fifo': 0,
                   'haircut': 0,
                   'poison': False,
                   'seniority': 0},
                  {'address': '0xb16a310f518291a2661719a2b099a01b904f2d25',
                   'fifo': 0,
                   'haircut': 0,
                   'poison': False,
                   'seniority': 0},
                  {'address': '0x16e702864c5ecb63fff26b7417bbe34a1702497f',
                   'fifo': 0,
                   'haircut': 0,
                   'poison': False,
                   'seniority': 8.425725699201642},
                  {'address': '0x5a891155f50e42074374c739baadf7df2651153a',
                   'fifo': 0,
                   'ha

# Produce one final value!

In [28]:
def cluster_taint_level(cluster: dict, exclude: list[str] = []):
    """
    Calculates the taint level of a cluster.
    """
    SENIORITY_WEIGHT = 0.5
    POISON_WEIGHT = 0.25
    HAIRCUT_WEIGHT = 0.125
    FIFO_WEIGHT = 0.125

    taint_level = 0
    for node in cluster['blacklisting']:
        if node['address'] in exclude:
            continue
        taint_level += node['seniority'] * SENIORITY_WEIGHT
        taint_level += 1 * POISON_WEIGHT if node['poison'] else 0
        taint_level += node['haircut'] * HAIRCUT_WEIGHT
        taint_level += node['fifo'] * FIFO_WEIGHT
    return taint_level

def individual_taint_level(node: dict):
    """
    Calculates the taint level of an individual node.
    """
    SENIORITY_WEIGHT = 0.5
    POISON_WEIGHT = 0.25
    HAIRCUT_WEIGHT = 0.125
    FIFO_WEIGHT = 0.125

    taint_level = 0
    taint_level += node['seniority'] * SENIORITY_WEIGHT
    taint_level += 1 * POISON_WEIGHT if node['poison'] else 0
    taint_level += node['haircut'] * HAIRCUT_WEIGHT
    taint_level += node['fifo'] * FIFO_WEIGHT
    return taint_level

async def calculate_risk_level(address: str):
    combined_cluster_data = await get_cluster_with_blacklisting_data(address)
    if combined_cluster_data is None:
        return None
    cluster_taint = cluster_taint_level(combined_cluster_data, exclude=[address])

    individual_data = {}
    for node in combined_cluster_data['blacklisting']:
        if node['address'] == address:
            individual_data = node
            break
    individual_taint = individual_taint_level(individual_data)

    INDIVIDUAL_WEIGHT = 0.8
    CLUSTER_WEIGHT = 0.2
    return {
        "cluster_taint": cluster_taint,
        "individual_taint": individual_taint,
        "risk_level": cluster_taint * CLUSTER_WEIGHT + individual_taint * INDIVIDUAL_WEIGHT,
    }
    

In [29]:
pprint(await calculate_risk_level('0x2f86e77d4e577d32f5d1d51289f80cfc95b4f460'))

{'cluster_taint': 4.533479200290964,
 'individual_taint': 0.0,
 'risk_level': 0.9066958400581928}
